# Generating Word Embeddings - Lab

## Introduction

In this lab, we'll learn how to generate our own word embeddings by training our own Word2Vec model, and also by building embedding layers right into our Deep Neural Networks!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, we'll start by creating our own word embeddings by making use of the Word2Vec Model. Then, we'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in our model. 

The easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which we can use to begin training immediately. For this lab, we'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.  In this lab, we'll learn how to train a Word2Vec model on the text data to generate word embeddings for them. In the next lab, we'll then use the vectors created by our Word2Vec model to effectively train a classifier to predict the category of news given the headline and description of each article. In this lab, we won't do any classification, although we will learn how to train a Word2Vec model and explore the relationships between different word vectors in our embedding!

Run the cell below to import everything we'll need for this lab. 

In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, we'll import the data. You'll find the data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to also include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what our data looks like. 

In [7]:
df = pd.read_json('News_Category_Dataset_v2.json', lines = True)

In [8]:
df.head

<bound method NDFrame.head of              category                                           headline  \
0               CRIME  There Were 2 Mass Shootings In Texas Last Week...   
1       ENTERTAINMENT  Will Smith Joins Diplo And Nicky Jam For The 2...   
2       ENTERTAINMENT    Hugh Grant Marries For The First Time At Age 57   
3       ENTERTAINMENT  Jim Carrey Blasts 'Castrato' Adam Schiff And D...   
4       ENTERTAINMENT  Julianna Margulies Uses Donald Trump Poop Bags...   
...               ...                                                ...   
200848           TECH  RIM CEO Thorsten Heins' 'Significant' Plans Fo...   
200849         SPORTS  Maria Sharapova Stunned By Victoria Azarenka I...   
200850         SPORTS  Giants Over Patriots, Jets Over Colts Among  M...   
200851         SPORTS  Aldon Smith Arrested: 49ers Linebacker Busted ...   
200852         SPORTS  Dwight Howard Rips Teammates After Magic Loss ...   

                 authors                                 

## Preparing the Data

Since we're working with text data, we'll still need to do some basic preprocessing and tokenize our data. You'll notice from the sample of the data above that two different columns contain text data--`headline` and `short_description`. The more text data our Word2Vec model has, the better it will perform. Therefore, we'll want to combine the two columns before tokenizing each comment and training our Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [10]:
df.short_description

0         She left her husband. He killed their children...
1                                  Of course it has a song.
2         The actor and his longtime girlfriend Anna Ebe...
3         The actor gives Dems an ass-kicking for not fi...
4         The "Dietland" actress said using the bags is ...
                                ...                        
200848    Verizon Wireless and AT&T are already promotin...
200849    Afterward, Azarenka, more effusive with the pr...
200850    Leading up to Super Bowl XLVI, the most talked...
200851    CORRECTION: An earlier version of this story i...
200852    The five-time all-star center tore into his te...
Name: short_description, Length: 200853, dtype: object

In [9]:
df.headline

0         There Were 2 Mass Shootings In Texas Last Week...
1         Will Smith Joins Diplo And Nicky Jam For The 2...
2           Hugh Grant Marries For The First Time At Age 57
3         Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4         Julianna Margulies Uses Donald Trump Poop Bags...
                                ...                        
200848    RIM CEO Thorsten Heins' 'Significant' Plans Fo...
200849    Maria Sharapova Stunned By Victoria Azarenka I...
200850    Giants Over Patriots, Jets Over Colts Among  M...
200851    Aldon Smith Arrested: 49ers Linebacker Busted ...
200852    Dwight Howard Rips Teammates After Magic Loss ...
Name: headline, Length: 200853, dtype: object

In [11]:
df['combined_text'] = df.headline + ' ' + df.short_description
data = df['combined_text'].map(word_tokenize)

Let's inspect the first 5 items in `data` to see how everything looks. 

In [12]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

You'll notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Recall from our previous lesson on how Word2Vec works that we can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If our window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example we saw).

Now that we've prepared our data, let's train our model and explore a bit!

## Training the Model

We'll start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [13]:
model = Word2Vec(data, size = 100, window = 5, min_count = 1, workers = 4)

Now, that we've created our Word2Vec model, we still need to train it on our model. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which we can find in `model.corpus_count`. 
    * The number of `epochs` we want to train for, which we'll set to `10`

In [14]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55565320, 67352790)

Great! We now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which we'll find stored inside of `model.wv`. For simplicity's sake, let's go ahead and store this inside of the variable `wv` in order to save ourselves some keystrokes down the line. 

In [15]:
wv = model.wv

## Examining Our Word Vectors

Now that we have a trained Word2Vec model, let's go ahead and explore the relationships between some of the words in our corpus! 

One cool thing we can use Word2Vec for is to get the most similar words to a given word. We can do this passing in the word to `wv.most_similar()`. 

In the cell below, let's try getting the most similar word to `'Texas'`.

In [16]:
wv.most_similar('Texas')

[('Louisiana', 0.8141637444496155),
 ('Ohio', 0.8064354658126831),
 ('Oregon', 0.8054990768432617),
 ('Florida', 0.7918329238891602),
 ('Georgia', 0.7902688980102539),
 ('Wisconsin', 0.7893681526184082),
 ('Pennsylvania', 0.7863590717315674),
 ('Maryland', 0.7855081558227539),
 ('Connecticut', 0.7854250073432922),
 ('Michigan', 0.7829806804656982)]

Interesting! All of the most similar words are also states. 

We can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [17]:
wv.most_similar(negative = 'Texas')

[('uh-oh', 0.4472462236881256),
 ('once-reliable', 0.41502809524536133),
 ('Parent/Grandparent', 0.41485369205474854),
 ('much-vaunted', 0.405956506729126),
 ('Headstrong', 0.4049034118652344),
 ('Slammers', 0.36311471462249756),
 ('Hunger-Free', 0.36197370290756226),
 ('Out-Crazy', 0.3612590730190277),
 ('man-children', 0.35864484310150146),
 ('maitre', 0.35777023434638977)]

These seem like just noise. This is because of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with our given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

We can also get the vector for a given word by passing in the word as if we were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [18]:
wv['Texas']

array([ 1.0751716 , -0.9239829 ,  0.9736828 , -0.78803605, -0.35479766,
       -1.6081524 , -0.39382946, -2.335709  , -2.4321294 , -1.3434573 ,
        0.7456731 , -0.8332664 ,  2.0616972 , -2.28709   ,  2.912695  ,
       -0.08026685, -0.10357848,  1.075434  ,  0.08985163, -0.69731843,
       -0.8373447 ,  0.9418225 ,  0.13095362,  0.62850875, -1.2525632 ,
        0.60992754, -1.5785917 ,  1.5212094 , -0.38330227,  1.9674778 ,
        1.017183  ,  1.262556  , -0.9156039 ,  1.4380486 ,  0.6753055 ,
        1.428953  ,  0.26687738, -2.5812216 ,  0.7699734 , -1.5147856 ,
       -1.5447688 , -1.6239151 , -0.21134897, -1.0671902 , -0.45782474,
       -0.12694694, -0.6696905 , -0.5725269 , -0.24149232,  0.5652009 ,
        2.113911  , -0.40139467,  0.6568888 ,  0.08170167,  1.2212299 ,
        2.7719035 ,  1.1305711 ,  1.8311994 , -0.9866743 , -0.2372598 ,
       -1.034475  ,  0.56085366, -0.49702722, -2.5329754 ,  1.9825127 ,
       -0.4329552 ,  1.3751969 , -0.10637812,  1.2775933 ,  1.15

Let's get all of the word vectors from the object at once. We can find these inside of `wv.vectors`.  Do this now in the cell below.  

In [19]:
wv.vectors

array([[ 2.68644547e+00,  1.17436612e+00,  1.05622768e+00, ...,
         7.56933510e-01,  1.16852307e+00,  1.87648427e+00],
       [ 2.62112093e+00,  9.11401987e-01,  1.00231397e+00, ...,
         1.72116673e+00,  2.12874866e+00,  2.72196460e+00],
       [ 2.55948091e+00,  1.39388412e-01, -1.51728404e+00, ...,
         1.58396268e+00,  7.58419752e-01, -1.76212549e+00],
       ...,
       [-3.20773548e-03, -8.09711590e-02,  2.26045190e-03, ...,
         3.76693979e-02, -5.40859066e-02, -1.90120656e-02],
       [ 1.14288535e-02, -1.12896472e-01,  7.96830934e-03, ...,
         7.31190946e-03,  1.70537345e-02,  1.09326346e-02],
       [-4.07261178e-02, -2.26045512e-02, -6.54375320e-03, ...,
        -2.99007222e-02, -6.98295608e-02,  3.25992256e-02]], dtype=float32)

As a final exercise, let's try recreating the _'king' - 'man' + 'woman' = 'queen'_ example we've seen before. We can do this by using the `most_similar` function and putting the things we want added together inside of an array passed to the `positive` parameter, and the things we want subtracted as an array passed to the the `negative` parameter. 

Do this now in the cell below. 

In [21]:
wv.most_similar(positive = ['king', 'woman'], negative = ['man'])

[('princess', 0.6432393789291382),
 ('title', 0.6389867067337036),
 ('brunette', 0.6075875759124756),
 ('villain', 0.5817644596099854),
 ('supermodel', 0.5786864757537842),
 ('revival', 0.5662742853164673),
 ('invention', 0.5584447383880615),
 ('centerpiece', 0.5559278130531311),
 ('dancer', 0.5554729700088501),
 ('diva', 0.5531613826751709)]

As we can see from the output above, our model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As we can see from the word in first place, 'reminiscent', our model is far from perfect. This is likely because we didn't give it too much training, or training data. However, for the small amount of training data it was given, the model still performs remarkably well! 

We'll see in the next lab that from a practical standpoint, one of the best things we can do for performance is to start by loading in the weights from an open-sourced model that has been trained for a very long time on a massive amount of data, such as the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless our text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, we learned how to train and use a Word2Vec model to created vectorized word embeddings!